# EDA в NLP

## Зависимости

In [ ]:
!pip install datasets==2.1.0 seaborn==0.11.2 scikit-learn==1.0.2 gensim==4.2.0 nltk==3.7 pymystem3==0.2.0 pyLDAvis==3.3.1


In [ ]:

from datasets import load_dataset
from gensim import corpora
from gensim.models.ldamodel import LdaModel
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer 
from pymystem3 import Mystem
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import nltk
import numpy as np
import pandas as pd
import seaborn as sns

from string import punctuation

#Create lemmatizer and stopwords list
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")

nltk.download("stopwords")

## Статистика по датасету

In [ ]:
# Загружаем датасет

dataset = load_dataset("blinoff/kinopoisk")

In [ ]:
i = np.random.randint(0, len(dataset["train"]))

print(dataset["train"][i]["movie_name"], dataset["train"][i]["grade3"])
print(dataset["train"][i]["content"])


## Предобработка

### Стемминг

In [ ]:
stemmer = SnowballStemmer("russian") 


def stem_text(text):
    tokens = text.lower()
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    tokens = text.lower().split()
    
    return " ".join(list(map(stemmer.stem, tokens)))

stem_text("Ну что сказать, я вижу кто-то наступил на грабли, Ты разочаровал меня, ты был натравлен.")

### Лемматизация

In [ ]:
def lemmatize_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords \
              and token != " " \
              and token.strip() not in punctuation]
    
    text = " ".join(tokens)
    
    return text

lemmatize_text("Ну что сказать, я вижу кто-то наступил на грабли, Ты разочаровал меня, ты был натравлен.")

In [ ]:
from typing import List

def flatten(sentences: List[List[str]]) -> List[str]:
    result = []
    for sentence in sentences:
        result += sentence
    return result


words = flatten(map(lambda text: text.split(), dataset['train']["content"]))
n_words = len(words)
n_unique_words = len(set(words))

print("{:,} кол-во примеров".format(len(dataset["train"])))
print("{:,} слов".format(n_words))
print("{:,} уникальных слов".format(n_unique_words))

In [ ]:
# TODO: вывести кол-во уникальных лемматизированных слов


## Визуализация

In [ ]:
import pandas as pd
sentences_length_list = list(map(lambda text: len(text.split()), dataset['train']["content"]))

sns.distplot(
    pd.Series(sentences_length_list, name="Кол-во слов"),
    label="Распределение кол-ва слов",
    kde=False
)


In [ ]:
sentences_count_list = list(map(lambda text: len(text.split(". ")), dataset['train']["content"]))

sns.distplot(
    pd.Series(sentences_count_list, name="Кол-во предложений"),
    label="Распределение кол-ва слов",
    kde=False
)


In [ ]:
# TODO: вывести распределение кол-ва слов по каждому классу


## ТОПы

In [ ]:
# Топ 50 часто используемых слов
pd.Series(words).value_counts().head(50)

In [ ]:
# TODO: вывести топ-50 часто используемых слов без учета союзов и предлогов


In [ ]:
df = dataset["train"].to_pandas()

pd.Series(flatten(df.loc[df["grade3"] == "Good", "content"].str.strip().apply(lambda t: t.split()))).value_counts().head()

In [ ]:
# TODO: вывести топ уникальных слов для каждого класса

## Наиболее значимые слова

In [ ]:
tfidf = TfidfVectorizer().fit(dataset["train"]["content"])
logreg = LogisticRegression().fit(tfidf.transform(dataset["train"]["content"]), dataset["train"]["grade3"])


In [ ]:
logreg.coef_.shape, len(tfidf.vocabulary_)


In [ ]:
itos = {i: v for v, i in tfidf.vocabulary_.items()}

In [ ]:
class_index = 1

print("Наиболее значимые слова для класса:", logreg.classes_[class_index])
for i in np.abs(logreg.coef_[class_index]).argsort()[-20:]:
    print("-", itos[i])


In [ ]:
# TODO: вывести наиболее значимые слова с лемматизацией

## Тематические модели

In [ ]:

def remove_stop_words(text: List[str]) -> List[str]:
    return [word for word in text if word not in russian_stopwords]


In [ ]:
words = list(map(lambda t: remove_stop_words(t.lower().split()), dataset["train"]["content"]))
dictionary = corpora.Dictionary(words)

In [ ]:
corpus = [dictionary.doc2bow(t) for t in words]

lda_model = LdaModel(
    corpus=corpus, id2word=dictionary, num_topics=3, iterations=100, alpha='auto')

In [ ]:
from pprint import pprint
pprint(lda_model.print_topics())

In [ ]:
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()

pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)